In [46]:
import pandas as pd
import os

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from datetime import datetime
from pyspark.sql.functions import col,udf
from pyspark.sql.types import DateType
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)



In [34]:

fulldata = pd.read_csv(os.environ.get("TAXI_DATA"))
weatherdata = pd.read_excel(os.environ.get("WEATHER_DATA"))
holidaysdata = pd.read_excel(os.environ.get("HOLIDAY_DATA"))


In [ ]:
raw_fulldata = spark.createDataFrame(pd.read_csv(os.environ.get("TAXI_DATA")))
raw_weatherdata = spark.createDataFrame(pd.read_excel(os.environ.get("WEATHER_DATA")))
raw_holidaysdata = spark.createDataFrame(pd.read_excel(os.environ.get("HOLIDAY_DATA")))

In [ ]:
raw_fulldata.printSchema()

In [ ]:
raw_weatherdata.printSchema()

In [35]:
raw_holidaysdata.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Holiday: string (nullable = true)



In [36]:
raw_holidaysdata.show()
#raw_holidaysdata_sample_df.show()

+--------+--------------------+
|    Date|             Holiday|
+--------+--------------------+
|01.01.15|       New Years Day|
|19.01.15|Martin Luther Kin...|
|12.02.15|  Lincoln's Birthday|
|16.02.15|     Presidents' Day|
|10.05.15|        Mother's Day|
|25.05.15|        Memorial Day|
|21.06.15|        Father's Day|
|03.07.15|Independence Day ...|
|07.09.15|           Labor Day|
|12.10.15|        Columbus Day|
|11.11.15|        Veterans Day|
|26.11.15|        Thanksgiving|
|27.11.15|Day after Thanksg...|
|25.12.15|       Christmas Day|
+--------+--------------------+



In [37]:
raw_weatherdata.show()

+-----------+----------+----+---+----+
|pickup_date|avg_temp_C|Rain|Fog|Snow|
+-----------+----------+----+---+----+
|   01.01.15|         1| NaN|NaN| NaN|
|   02.01.15|         4| NaN|NaN| NaN|
|   03.01.15|         3| 1.0|NaN| 1.0|
|   04.01.15|         9| 1.0|NaN| NaN|
|   05.01.15|         2| NaN|NaN| NaN|
|   06.01.15|        -6| NaN|NaN| 1.0|
|   07.01.15|        -9| NaN|NaN| NaN|
|   08.01.15|        -9| NaN|NaN| NaN|
|   09.01.15|        -3| NaN|NaN| 1.0|
|   10.01.15|        -7| NaN|NaN| NaN|
|   11.01.15|        -2| NaN|NaN| NaN|
|   12.01.15|         3| 1.0|NaN| NaN|
|   13.01.15|        -3| NaN|NaN| NaN|
|   14.01.15|        -4| NaN|NaN| NaN|
|   15.01.15|        -1| NaN|NaN| NaN|
|   16.01.15|         0| NaN|NaN| NaN|
|   17.01.15|        -4| NaN|NaN| NaN|
|   18.01.15|         3| 1.0|1.0| NaN|
|   19.01.15|         4| NaN|NaN| NaN|
|   20.01.15|         2| NaN|NaN| NaN|
+-----------+----------+----+---+----+
only showing top 20 rows



In [38]:
raw_fulldata.limit(20).show(3)

+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------------+--------------------+-----------+---------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|pickup_zip|pickup_borough| pickup_neighborhood|dropoff_zip|dropoff_borough|dropoff_neighborhood|
+--------+-------------------+-------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+-----

In [39]:
fulldata.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,0.00,0.0,0.3,9.80,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,2.45,0.0,0.3,10.75,10023,Manhattan,Upper West Side,10024,Manhattan,Upper West Side
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,1.00,0.0,0.3,11.80,10010,Manhattan,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,0.00,0.0,0.3,13.30,10065,Manhattan,Upper East Side,10023,Manhattan,Upper West Side
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,0.00,0.0,0.3,7.30,10065,Manhattan,Upper East Side,10019,Manhattan,Chelsea and Clinton


In [47]:
rdd = sc.parallelize(['01.01.15', '19.01.15', '12.02.15', '16.02.15'])
df1 = spark.createDataFrame(rdd, ['Date'])

#UDF to convert string to date
func =  udf (lambda x: datetime.strptime(x, '%d.%b.%Y'), DateType())

df = df1.withColumn('new_Date', date_format(func(col('Date')), 'MM-dd-yyy'))

df.show()

TypeError: Can not infer schema for type: <class 'str'>

In [11]:

datefrompickup=pd.to_datetime(fulldata['pickup_datetime']).apply(lambda x: x.date())
datefrompickup.head()
fulldata['pickup_date']=datefrompickup
fulldata.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,tolls_amount,improvement_surcharge,total_amount,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,0.0,0.3,9.80,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,2015-01-06
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,0.0,0.3,10.75,10023,Manhattan,Upper West Side,10024,Manhattan,Upper West Side,2015-01-13
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,0.0,0.3,11.80,10010,Manhattan,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton,2015-01-16
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,0.0,0.3,13.30,10065,Manhattan,Upper East Side,10023,Manhattan,Upper West Side,2015-01-23
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,0.0,0.3,7.30,10065,Manhattan,Upper East Side,10019,Manhattan,Chelsea and Clinton,2015-01-24


In [12]:
mergeddata=fulldata.join(weatherdata,lsuffix='_fulldata', rsuffix='_weatherdata')
mergeddata.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date_fulldata,pickup_date_weatherdata,avg_temp_C,Rain,Fog,Snow
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,2015-01-06,01.01.15,1.0,NaN,NaN,NaN
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,Upper West Side,10024,Manhattan,Upper West Side,2015-01-13,02.01.15,4.0,NaN,NaN,NaN
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton,2015-01-16,03.01.15,3.0,1.0,NaN,1.0
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,Upper East Side,10023,Manhattan,Upper West Side,2015-01-23,04.01.15,9.0,1.0,NaN,NaN
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,Upper East Side,10019,Manhattan,Chelsea and Clinton,2015-01-24,05.01.15,2.0,NaN,NaN,NaN


In [13]:
fulldata.merge(weatherdata, left_on='pickup_date',right_on='pickup_date',how='inner')

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,pickup_date,avg_temp_C,Rain,Fog,Snow


In [14]:
weatherdata.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow
0,01.01.15,1,NaN,NaN,NaN
1,02.01.15,4,NaN,NaN,NaN
2,03.01.15,3,1.0,NaN,1.0
3,04.01.15,9,1.0,NaN,NaN
4,05.01.15,2,NaN,NaN,NaN


In [15]:

weatherdata['pickup_date'] = pd.to_datetime(weatherdata['pickup_date'], format="%d.%m.%y", errors='coerce')
weatherdata.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow
0,2015-01-01,1,NaN,NaN,NaN
1,2015-01-02,4,NaN,NaN,NaN
2,2015-01-03,3,1.0,NaN,1.0
3,2015-01-04,9,1.0,NaN,NaN
4,2015-01-05,2,NaN,NaN,NaN


In [16]:
weatherdata.tail()

,pickup_date,avg_temp_C,Rain,Fog,Snow
360,2015-12-27,12,1.0,NaN,NaN
361,2015-12-28,4,1.0,NaN,NaN
362,2015-12-29,4,1.0,NaN,NaN
363,2015-12-30,6,1.0,NaN,NaN
364,2015-12-31,7,1.0,NaN,NaN
